In [ ]:
!unzip /content/sample_data/Indicators_of_Heart_Disease_2022.zip -d /content/Indicators_of_Heart_Disease_2022_UPDATE


unzip:  cannot find or open /content/sample_data/Indicators_of_Heart_Disease_2022.zip, /content/sample_data/Indicators_of_Heart_Disease_2022.zip.zip or /content/sample_data/Indicators_of_Heart_Disease_2022.zip.ZIP.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/sample_data/heart_2022_no_nans.csv')
print(df.head())


     State     Sex GeneralHealth  PhysicalHealthDays  MentalHealthDays  \
0  Alabama  Female     Very good                 4.0               0.0   
1  Alabama    Male     Very good                 0.0               0.0   
2  Alabama    Male     Very good                 0.0               0.0   
3  Alabama  Female          Fair                 5.0               0.0   
4  Alabama  Female          Good                 3.0              15.0   

                                     LastCheckupTime PhysicalActivities  \
0  Within past year (anytime less than 12 months ...                Yes   
1  Within past year (anytime less than 12 months ...                Yes   
2  Within past year (anytime less than 12 months ...                 No   
3  Within past year (anytime less than 12 months ...                Yes   
4  Within past year (anytime less than 12 months ...                Yes   

   SleepHours            RemovedTeeth HadHeartAttack  ... HeightInMeters  \
0         9.0            Non

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# Load the data
df = pd.read_csv('/content/sample_data/heart_2022_no_nans.csv')

# Identify columns with object type (likely categorical)
categorical_cols = df.select_dtypes(include=["object"]).columns

# Encode categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Define features (X) and target (y)
y = df['HighRiskLastYear']  # Adjust target variable as needed
# Get the correct feature names from the DataFrame's columns
features = df.columns.tolist()
features.remove('HighRiskLastYear')
X = df[features]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#initalizer random forestregressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


#make prediction
predictions = model.predict(X_test)

#evaluate model
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.08297510415608168


In [ ]:
# Calculate accuracy (assuming a threshold for classification)
threshold = 0.5  # Example threshold, adjust as needed
predicted_classes = (predictions > threshold).astype(int)
accuracy = (predicted_classes == y_test).mean()
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.96


In [ ]:
print("Predictions")
predictions

Predictions


array([0.  , 0.01, 0.12, ..., 0.18, 0.01, 0.2 ])

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error

# Model data
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
val_mae = mean_absolute_error(y_pred, y_test)
print(f"Validation MAE without max_leaf_nodes: {val_mae}")
accuracy = accuracy_score(y_test, y_pred.round())
print(f"Accuracy: {accuracy:.2f}")

# Model with max_leaf_nodes
model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
val_mae = mean_absolute_error(y_pred, y_test)
print(f"Validation MAE with 100 max_leaf_nodes: {val_mae}")
accuracy = accuracy_score(y_test, y_pred.round())
print(f"Accuracy: {accuracy:.2f}")


Validation MAE without max_leaf_nodes: 0.08316228025607154
Accuracy: 0.92
Validation MAE with 100 max_leaf_nodes: 0.07396823352164747
Accuracy: 0.96


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import os
import logging
import joblib
from datetime import datetime


In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],      # 500, 700, 1000: Increased range of trees
    'max_depth': [10, 20, 30, 40, None], # Added more depth levels
    'min_samples_split': [2, 5, 10],     # Removed unnecessary high values
    'min_samples_leaf': [1, 2, 4],       # Reduced high values
    'max_features': ['none', 'sqrt', 'log2'], # Focused features
    'bootstrap': [True, False],          # Keeping both for flexibility
    'criterion': ['squared_error']       # Focus on squared error for regression
}


In [ ]:
# Initialize the model
model = RandomForestRegressor(random_state=42)

# Apply GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
history = grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 810 candidates, totalling 4050 fits


KeyboardInterrupt: 

In [ ]:
# Best model and evaluation
best_model = grid_search.best_estimator_

# Save the best model
joblib.dump(best_model, os.path.join('./', "best_gridsearch_param_pred.pkl"))


In [ ]:
joblib.dump(best_model, os.path.join('./', "best_gridsearch_param_pred.pkl"))


NameError: name 'best_model' is not defined

In [ ]:
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Confidence (based on mean absolute error)
confidence = 100 * (1 - mae / y.max())

print(f"Final Model Evaluation - MSE: {mse:.4f}, R2: {r2:.4f}")
print(f"Average Confidence: {confidence:.2f}%")


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred)
plt.xlabel("Actual Output")
plt.ylabel("Predicted Output")
plt.title("Actual vs. Predicted Output")
plt.show()


In [ ]:
y_pred_discrete = np.round(y_pred).astype(int)
cm = confusion_matrix(y_test, y_pred_discrete)

plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
